# Chapter 8
# Data Aggregation and Group Operations

Categorizing a dataset and applying a function to each group, whether an aggregation or transformation, is often a critical component of a data analysis workflow. After loading, merging, and preparing a dataset, you may need to compute group statistics or possibly pivot tables for reporting or visualization purposes. pandas provides a flexible groupby interface, enabling you to slice, dice, and summarize datasets in a natural way.

One reason for the popularity of relational databases and SQL (which stands for “structured query language”) is the ease with which data can be joined, filtered, transformed, and aggregated. However, query languages like SQL are somewhat constrained in the kinds of group operations that can be performed. As you will see, with the expressiveness of Python and pandas, we can perform quite complex group operations by utilizing any function that accepts a pandas object or NumPy array.

## 8.1 GroupBy Mechanics

Hadley Wickham, an author of many popular packages for the R programming language, coined the term split-apply-combine for describing group operations. In the first stage of the process, data contained in a pandas object, whether a Series, Data‐
Frame, or otherwise, is split into groups based on one or more keys that you provide. The splitting is performed on a particular axis of an object. 

For example, a DataFrame can be grouped on its <b>rows *(axis=0)*</b> or its <b>columns *(axis=1)*</b>. Once this is done, a function is applied to each group, producing a new value. Finally, the results of all those function applications are combined into a result object. The form of the resulting object will usually depend on what’s being done to the data. See Figure 8-1 for a
mockup of a simple group aggregation.

<img src="Figure 8.1.png" width="600px">
<br>
<center>Figure 8.1: Illustration of a group aggregation</center>

Each grouping key can take many forms, and the keys do not have to be all of the same type:<br>
• A list or array of values that is the same length as the axis being grouped <br>
• A value indicating a column name in a DataFrame <br>
• A dict or Series giving a correspondence between the values on the axis being grouped and the group names <br>
• A function to be invoked on the axis index or the individual labels in the index <br>

Note that the latter three methods are shortcuts for producing an array of values to be used to split up the object.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'], 'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5), 'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.069972,0.098014
1,a,two,1.835424,0.721582
2,b,one,0.839425,-0.960271
3,b,two,0.518822,-1.087114
4,a,one,-0.103172,-1.593120


Suppose you wanted to compute the mean of the data1 column using the labels from *key1*. There are a number of ways to do this. One is to access data1 and call `groupby` with the column (a Series) at key1:

In [3]:
df['key1']

0    a
1    a
2    b
3    b
4    a
Name: key1, dtype: object

In [4]:
grouped = df['data1'].groupby(df['key1'])
list(grouped)

[('a',
  0   -0.069972
  1    1.835424
  4   -0.103172
  Name: data1, dtype: float64),
 ('b',
  2    0.839425
  3    0.518822
  Name: data1, dtype: float64)]

This grouped variable is now a *GroupBy* object. It has not actually computed anything yet except for some intermediate data about the group key `df['key1']`. The idea is that this object has all of the information needed to then apply some operation to
each of the groups. For example, to compute group means we can call the GroupBy’s `mean` method:

<br>
<img src="Example1.png"/>

In [6]:
ans = grouped.sum()
# df['data1'].groupby(df['key1']).mean()
ans

key1
a    1.662280
b    1.358246
Name: data1, dtype: float64

The important thing here is that the data (a Series) has been aggregated according to the group key, producing a new Series that is now indexed by the unique values in the key1 column.

The result index has the name `'key1'` because the DataFrame column `df['key1']` did.

If instead we had passed multiple arrays as a list, we’d get something different:
<br>
<img src="Example2.png"/>

In [12]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.086572
      two     1.835424
b     one     0.839425
      two     0.518822
Name: data1, dtype: float64

Here we grouped the data using two keys, and the resulting Series now has a hierarchical index consisting of the unique pairs of keys observed:

In [13]:
means_2d = means.unstack()
means_2d

key2,one,two
key1,,
a,-0.086572,1.835424
b,0.839425,0.518822


In this example, the group keys are all Series, though they could be any arrays of the right length:

In [14]:
states = np.array(['Ohio', 'Ohio', 'California', 'Ohio', 'Ohio'])
# years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states]).mean()

California    0.839425
Ohio          0.545275
Name: data1, dtype: float64

Frequently the grouping information is found in the same DataFrame as the data you want to work on. In that case, you can pass column names (whether those are strings, numbers, or other Python objects) as the group keys:


In [15]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.554093,-0.257841
b,0.679123,-1.023693


In [16]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.086572 -0.747553
     two   1.835424  0.721582
b    one   0.839425 -0.960271
     two   0.518822 -1.087114

You may have noticed in the first case `df.groupby('key1').mean()` that there is no `key2` column in the result. Because `df['key2']` is not numeric data, it is said to be a *nuisance column*, which is therefore excluded from the result. <b>By default, all of the numeric columns are aggregated</b>, though it is possible to filter down to a subset.

Regardless of the objective in using groupby, a generally useful GroupBy method is `size`, which returns a Series containing group sizes:

<br>
<img src='Example3.png'/>

In [18]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

Take note that any missing values in a group key will be **excluded** from the result.

In [19]:
df1 = pd.DataFrame({'k1' : ['a', 'a', 'b', 'b', 'a'], 'k2' : ['one', 'two', 'one', 'two', 'one'],
                   'd1' : np.random.randn(5), 'd2' : np.random.randn(5)})
df1['d1'][0]=""
df1.groupby(['k1','k2']).mean()

<ipython-input-19-dcce55101b92>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['d1'][0]=""
C:\Users\Junyao\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


d2
k1 k2           
a  one -1.567174
   two -1.217204
b  one -0.227005
   two  0.739449

In [20]:
df1

,k1,k2,d1,d2
0,a,one,,-1.317491
1,a,two,-1.83443,-1.217204
2,b,one,0.412716,-0.227005
3,b,two,-0.0265165,0.739449
4,a,one,-2.41694,-1.816856


### 8.1.1 Iterating Over Groups

The GroupBy object supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data. Consider the following:

In [21]:
grouped_1k = df.groupby('key1')
grouped_2k = df.groupby(['key1', 'key2'])
grouped_2k

In [22]:
for name, group in grouped_1k:
#     print(name)
    print(group)

  key1 key2     data1     data2
0    a  one -0.069972  0.098014
1    a  two  1.835424  0.721582
4    a  one -0.103172 -1.593120
  key1 key2     data1     data2
2    b  one  0.839425 -0.960271
3    b  two  0.518822 -1.087114


In the case of multiple keys, the first element in the tuple will be a tuple of key values:

In [23]:
for (k1, k2), group in grouped_2k:
    print(k1, k2)
    print(group)

a one
  key1 key2     data1     data2
0    a  one -0.069972  0.098014
4    a  one -0.103172 -1.593120
a two
  key1 key2     data1     data2
1    a  two  1.835424  0.721582
b one
  key1 key2     data1     data2
2    b  one  0.839425 -0.960271
b two
  key1 key2     data1     data2
3    b  two  0.518822 -1.087114


You can choose to do whatever you want with the pieces of data. A recipe you may find useful is computing a dict of the data pieces as a one-liner:

In [24]:
pieces = dict(list(grouped_1k))
pieces['a']

,key1,key2,data1,data2
0,a,one,-0.069972,0.098014
1,a,two,1.835424,0.721582
4,a,one,-0.103172,-1.593120


By default `groupby` groups on *axis=0*, but you can group on any of the other axes. For example, we could group the columns of our example `df` here by `dtype` like so:

In [25]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [26]:
grouped_by_type = df.groupby(df.dtypes, axis=1)
list(grouped)

[('a',
  0   -0.069972
  1    1.835424
  4   -0.103172
  Name: data1, dtype: float64),
 ('b',
  2    0.839425
  3    0.518822
  Name: data1, dtype: float64)]

We can print out the groups like so:

In [27]:
for dtype, group in grouped:
    print(dtype)
    print(group)

a
0   -0.069972
1    1.835424
4   -0.103172
Name: data1, dtype: float64
b
2    0.839425
3    0.518822
Name: data1, dtype: float64


### 8.1.2 Selecting a Column or Subset of Columns

Indexing a GroupBy object created from a DataFrame with a column name or array of column names has the effect of column subsetting for aggregation. This means that:

    df.groupby('key1')['data1']
    df.groupby('key1')[['data2']]

are syntactic sugar for:

    df['data1'].groupby(df['key1'])
    df[['data2']].groupby(df['key1'])

Especially for large datasets, it may be desirable to aggregate only a few columns. For example, in the preceding dataset, to compute means for just the data2 column and get the result as a DataFrame, we could write:

In [28]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.747553
     two   0.721582
b    one  -0.960271
     two  -1.087114

The object returned by this indexing operation is a <b>grouped DataFrame if a list or array is passed</b> or a <b>grouped Series if only a single column name is passed</b> as a scalar:

In [29]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [30]:
s_grouped.mean()

key1  key2
a     one    -0.747553
      two     0.721582
b     one    -0.960271
      two    -1.087114
Name: data2, dtype: float64

### 8.1.3 Grouping with Dicts and Series

Grouping information may exist in a form other than an array. Let’s consider another example DataFrame:

In [31]:
people = pd.DataFrame(np.random.randn(5, 5), columns=['a', 'b', 'c', 'd', 'e'], index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,0.114028,0.219391,-0.199390,0.615846,-1.158877
Steve,0.981555,-1.142902,0.495057,-2.044864,-1.901748
Wes,0.371799,NaN,NaN,-0.734229,-0.165008
Jim,-0.328827,-0.259607,0.761004,-0.558558,-0.551293
Travis,0.507590,1.278024,-1.899688,0.202322,0.882582


Now, suppose we have a group correspondence for the columns and want to sum together the columns by group:

In [32]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f' : 'orange'}

Now, we could construct an array from this dict to pass to `groupby`, but instead we can just pass the `dict` (we included the key 'f' to highlight that unused grouping keys are OK):

In [33]:
by_column = people.groupby(mapping, axis=1)
# by_column.sum()
dict(list(by_column))
by_column.sum()

,blue,red
Joe,0.416456,-0.825458
Steve,-1.549808,-2.063095
Wes,-0.734229,0.206791
Jim,0.202446,-1.139726
Travis,-1.697366,2.668196


The same functionality holds for Series, which can be viewed as a fixed-size mapping:

In [34]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [35]:
people.groupby(map_series, axis=1).sum()

,blue,red
Joe,0.416456,-0.825458
Steve,-1.549808,-2.063095
Wes,-0.734229,0.206791
Jim,0.202446,-1.139726
Travis,-1.697366,2.668196


### 8.1.4 Grouping with Functions

Using Python functions is a more generic way of defining a group mapping compared with a `dict` or `Series`. Any function passed as a group key will be <b>called once per index value, with the return values being used as the group names.</b> More concretely, consider the example `DataFrame` from the previous section, which has people’s first names as index values. 

Suppose we wanted to group by the length of the names; while we could compute an array of string lengths, it’s simpler to just pass the `len` function:

In [36]:
people

,a,b,c,d,e
Joe,0.114028,0.219391,-0.199390,0.615846,-1.158877
Steve,0.981555,-1.142902,0.495057,-2.044864,-1.901748
Wes,0.371799,NaN,NaN,-0.734229,-0.165008
Jim,-0.328827,-0.259607,0.761004,-0.558558,-0.551293
Travis,0.507590,1.278024,-1.899688,0.202322,0.882582


In [37]:
people.groupby(len).mean()

,a,b,c,d,e
3,0.052333,-0.020108,0.280807,-0.225647,-0.625059
5,0.981555,-1.142902,0.495057,-2.044864,-1.901748
6,0.507590,1.278024,-1.899688,0.202322,0.882582


In [38]:
dict(list(people.groupby(len)))

{3:             a         b         c         d         e
 Joe  0.114028  0.219391 -0.199390  0.615846 -1.158877
 Wes  0.371799       NaN       NaN -0.734229 -0.165008
 Jim -0.328827 -0.259607  0.761004 -0.558558 -0.551293,
 5:               a         b         c         d         e
 Steve  0.981555 -1.142902  0.495057 -2.044864 -1.901748,
 6:               a         b         c         d         e
 Travis  0.50759  1.278024 -1.899688  0.202322  0.882582}

Mixing functions with arrays, dicts, or Series is not a problem as everything gets converted to arrays internally:
<br>
<img src="Example4.png"/>

In [39]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).mean()

a         b         c         d         e
3 one  0.242913  0.219391 -0.199390 -0.059192 -0.661942
  two -0.328827 -0.259607  0.761004 -0.558558 -0.551293
5 one  0.981555 -1.142902  0.495057 -2.044864 -1.901748
6 two  0.507590  1.278024 -1.899688  0.202322  0.882582

### 8.1.5 Grouping by Index Levels

A final convenience for hierarchically indexed datasets is the ability to aggregate using one of the levels of an axis index. Let’s look at an example:

In [40]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], [1, 3, 5, 1, 3]], names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.494976 -1.947464  1.946726 -2.026669  0.893905
1     -0.209833  0.071362 -0.060893  0.122227  0.875594
2      2.279015 -0.591593  0.100653  1.836046 -0.149366
3     -0.550835  0.789116  1.173478  0.892849 -0.727831

To group by level, pass the level number or name using the `level` keyword:

In [41]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 8.2 Data Aggregation

Aggregations refer to any data transformation that produces scalar values from arrays. The preceding examples have used several of them, including `mean`, `count`, `min`, and `sum`. We may wonder what is going on when we invoke `mean()` on a GroupBy object. Many common aggregations, such as those found in Table 8.1, have optimized implementations. However, we are not limited to only this set of methods.

<br><center>Table 8.1: Optimized groupby methods</center>
<img src="Table 8.1.png" width="500px">

We can use aggregations of our own devising and additionally call any method that is also defined on the grouped object. For example, we might recall that `quantile` computes sample quantiles of a Series or a DataFrame’s columns.

While `quantile` is not explicitly implemented for GroupBy, it is a Series method and thus available for use. Internally, GroupBy efficiently slices up the Series, calls `piece.quantile(0.9)` for each piece, and then assembles those results together into the result object:

In [42]:
df

,key1,key2,data1,data2
0,a,one,-0.069972,0.098014
1,a,two,1.835424,0.721582
2,b,one,0.839425,-0.960271
3,b,two,0.518822,-1.087114
4,a,one,-0.103172,-1.593120


In [43]:
grouped = df.groupby('key1')
# grouped.quantile()
grouped['data1'].mean()

key1
a    0.554093
b    0.679123
Name: data1, dtype: float64

To use your own aggregation functions, pass any function that aggregates an array to the `aggregate` or `agg` method:

In [44]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.938596,2.314702
b,0.320603,0.126843


We may notice that some methods like describe also work, even though they are not aggregations, strictly speaking:

In [45]:
grouped['data1'].describe()

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,3.0,0.554093,1.109789,-0.103172,-0.086572,-0.069972,0.882726,1.835424
b,2.0,0.679123,0.226701,0.518822,0.598972,0.679123,0.759274,0.839425


### 8.2.1 Column-Wise and Multiple Function Application

Let’s go to the tipping dataset. After loading it with `read_csv`, we add a tipping percentage column *tip_pct*:

In [46]:
tips = pd.read_csv('tips.csv')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [47]:
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']

tips[:10]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240
6,8.77,2.00,Male,No,Sun,Dinner,2,0.228050
7,26.88,3.12,Male,No,Sun,Dinner,4,0.116071
8,15.04,1.96,Male,No,Sun,Dinner,2,0.130319
9,14.78,3.23,Male,No,Sun,Dinner,2,0.218539


As we’ve already seen, aggregating a Series or all of the columns of a DataFrame is a matter of using `aggregate` with the desired function or calling a method like `mean` or `std`. However, we may want to aggregate using a different function depending on the column, or multiple functions at once. Fortunately, this is possible to do, which we’ll illustrate through a number of examples. First, we’ll group the *tips* by day and *smoker*:

In [48]:
grouped = tips.groupby(['day', 'smoker'])
dict(list(grouped))

{('Fri',
  'No'):      total_bill   tip     sex smoker  day    time  size   tip_pct
 91        22.49  3.50    Male     No  Fri  Dinner     2  0.155625
 94        22.75  3.25  Female     No  Fri  Dinner     2  0.142857
 99        12.46  1.50    Male     No  Fri  Dinner     2  0.120385
 223       15.98  3.00  Female     No  Fri   Lunch     3  0.187735,
 ('Fri',
  'Yes'):      total_bill   tip     sex smoker  day    time  size   tip_pct
 90        28.97  3.00    Male    Yes  Fri  Dinner     2  0.103555
 92         5.75  1.00  Female    Yes  Fri  Dinner     2  0.173913
 93        16.32  4.30  Female    Yes  Fri  Dinner     2  0.263480
 95        40.17  4.73    Male    Yes  Fri  Dinner     4  0.117750
 96        27.28  4.00    Male    Yes  Fri  Dinner     2  0.146628
 97        12.03  1.50    Male    Yes  Fri  Dinner     2  0.124688
 98        21.01  3.00    Male    Yes  Fri  Dinner     2  0.142789
 100       11.35  2.50  Female    Yes  Fri  Dinner     2  0.220264
 101       15.38  3.00  Fe

In [49]:
grouped_pct = grouped['tip_pct']
dict(list(grouped_pct))

{('Fri',
  'No'): 91     0.155625
 94     0.142857
 99     0.120385
 223    0.187735
 Name: tip_pct, dtype: float64,
 ('Fri',
  'Yes'): 90     0.103555
 92     0.173913
 93     0.263480
 95     0.117750
 96     0.146628
 97     0.124688
 98     0.142789
 100    0.220264
 101    0.195059
 220    0.180921
 221    0.259314
 222    0.223776
 224    0.117735
 225    0.153657
 226    0.198216
 Name: tip_pct, dtype: float64,
 ('Sat',
  'No'): 19     0.162228
 20     0.227679
 21     0.135535
 22     0.141408
 23     0.192288
 24     0.160444
 25     0.131387
 26     0.149589
 27     0.157604
 28     0.198157
 29     0.152672
 30     0.151832
 31     0.136240
 32     0.199203
 33     0.118415
 34     0.183915
 35     0.149626
 36     0.122624
 37     0.181335
 38     0.123596
 39     0.159898
 40     0.139651
 57     0.056797
 59     0.139424
 64     0.150085
 65     0.156873
 66     0.150152
 68     0.099357
 70     0.163894
 71     0.175747
 74     0.149355
 75     0.118934
 104    0.195029


In [50]:
grouped_pct.agg(['sum','max','mean'])

sum       max      mean
day  smoker                              
Fri  No      0.606602  0.187735  0.151650
     Yes     2.621746  0.263480  0.174783
Sat  No      7.112145  0.291990  0.158048
     Yes     6.212055  0.325733  0.147906
Sun  No      9.126438  0.252672  0.160113
     Yes     3.557756  0.710345  0.187250
Thur No      7.213414  0.266312  0.160298
     Yes     2.785676  0.241255  0.163863

If you pass a list of functions or function names instead, you get back a DataFrame with column names taken from the functions:

In [51]:
grouped_pct.agg([('Average','mean'), ('Std dev','std'), ('Max-Min',peak_to_peak)])

Average   Std dev   Max-Min
day  smoker                              
Fri  No      0.151650  0.028123  0.067349
     Yes     0.174783  0.051293  0.159925
Sat  No      0.158048  0.039767  0.235193
     Yes     0.147906  0.061375  0.290095
Sun  No      0.160113  0.042347  0.193226
     Yes     0.187250  0.154134  0.644685
Thur No      0.160298  0.038774  0.193350
     Yes     0.163863  0.039389  0.151240

Here we passed a list of aggregation functions to `agg` to evaluate indepedently on the data groups.

We don’t need to accept the names that GroupBy gives to the columns; notably, `lambda` functions have the name `'<lambda>'`, which makes them hard to identify. Thus, if we pass a list of (name, function) tuples, the first element of each tuple will be used as the DataFrame column names (we can think of a list of 2-tuples as an ordered mapping):

In [52]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

With a DataFrame we have more options, as we can specify a list of functions to apply to all of the columns or different functions per column. To start, suppose we wanted to compute the same three statistics for the *tip_pct* and *total_bill*
columns:

In [53]:
functions = [('No. of tips','count'), ('Average Tips','mean'), ('Max tips','max')]


In [54]:
result = grouped[['tip_pct', 'total_bill']].agg(functions)
result

tip_pct                         total_bill               \
            No. of tips Average Tips  Max tips No. of tips Average Tips   
day  smoker                                                               
Fri  No               4     0.151650  0.187735           4    18.420000   
     Yes             15     0.174783  0.263480          15    16.813333   
Sat  No              45     0.158048  0.291990          45    19.661778   
     Yes             42     0.147906  0.325733          42    21.276667   
Sun  No              57     0.160113  0.252672          57    20.506667   
     Yes             19     0.187250  0.710345          19    24.120000   
Thur No              45     0.160298  0.266312          45    17.113111   
     Yes             17     0.163863  0.241255          17    19.190588   

                      
            Max tips  
day  smoker           
Fri  No        22.75  
     Yes       40.17  
Sat  No        48.33  
     Yes       50.81  
Sun  No        48.17  
     Yes       45.35  
Thur No        41.19  
     Yes       43.11

As we can see, the resulting DataFrame has hierarchical columns, the same as we would get aggregating each column separately and using `concat` to glue the results together using the column names as the `keys` argument:

In [55]:
result['total_bill']

No. of tips  Average Tips  Max tips
day  smoker                                     
Fri  No                4     18.420000     22.75
     Yes              15     16.813333     40.17
Sat  No               45     19.661778     48.33
     Yes              42     21.276667     50.81
Sun  No               57     20.506667     48.17
     Yes              19     24.120000     45.35
Thur No               45     17.113111     41.19
     Yes              17     19.190588     43.11

As before, a list of tuples with custom names can be passed:

In [56]:
ftuples = [('Average', 'mean'), ('Variance', np.var)]
grouped[['tip_pct', 'total_bill']].agg(ftuples)

tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

Now, suppose we wanted to apply potentially different functions to one or more of the columns. To do this, pass a dict to `agg` that contains a mapping of column names to any of the function specifications listed so far:

In [57]:
dict(list(grouped))

{('Fri',
  'No'):      total_bill   tip     sex smoker  day    time  size   tip_pct
 91        22.49  3.50    Male     No  Fri  Dinner     2  0.155625
 94        22.75  3.25  Female     No  Fri  Dinner     2  0.142857
 99        12.46  1.50    Male     No  Fri  Dinner     2  0.120385
 223       15.98  3.00  Female     No  Fri   Lunch     3  0.187735,
 ('Fri',
  'Yes'):      total_bill   tip     sex smoker  day    time  size   tip_pct
 90        28.97  3.00    Male    Yes  Fri  Dinner     2  0.103555
 92         5.75  1.00  Female    Yes  Fri  Dinner     2  0.173913
 93        16.32  4.30  Female    Yes  Fri  Dinner     2  0.263480
 95        40.17  4.73    Male    Yes  Fri  Dinner     4  0.117750
 96        27.28  4.00    Male    Yes  Fri  Dinner     2  0.146628
 97        12.03  1.50    Male    Yes  Fri  Dinner     2  0.124688
 98        21.01  3.00    Male    Yes  Fri  Dinner     2  0.142789
 100       11.35  2.50  Female    Yes  Fri  Dinner     2  0.220264
 101       15.38  3.00  Fe

In [58]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [59]:
ans = grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'], 'size' : 'sum'})
ans

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

A DataFrame will have hierarchical columns only if multiple functions are applied to at least one column.

### 8.2.2 Returning Aggregated Data Without Row Indexes

In all of the examples up until now, the aggregated data comes back with an index, potentially hierarchical, composed from the unique group key combinations. Since this isn’t always desirable, we can disable this behavior in most cases by passing
`as_index=False` to `groupby`:

In [60]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


Of course, it’s always possible to obtain the result in this format by calling `reset_index` on the result. Using the `as_index=False` method avoids some unnecessary computations.

## 8.3 Apply: General split-apply-combine

The most general-purpose GroupBy method is apply, which is the subject of the rest of this section. As illustrated in Figure 8.1 previously, apply splits the object being manipulated into pieces, invokes the passed function on each piece, and then attempts to concatenate the pieces together.

<img src="Figure 8.1.png" width="600px">
<br>
<center>Figure 8.1: Illustration of a group aggregation</center>

Returning to the tipping dataset from before, suppose we wanted to select the top five *tip_pct* values by group. First, write a function that selects the rows with the largest values in a particular column:

In [61]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips, n=6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


Now, if we group by *smoker*, say, and call apply with this function, we get the following:

In [62]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

What has happened here? The top function is called on each row group from the DataFrame, and then the results are glued together using `pandas.concat`, labeling the pieces with the group names. The result therefore has a hierarchical index whose
inner level contains index values from the original DataFrame.

If you pass a function to apply that takes other arguments or keywords, you can pass these after the function:

In [63]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   

                  tip_pct  
smoker day                 
No     Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799  
       Thur 142  0.121389  
Yes    Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
       Thur 197  0.115982

We may recall that we earlier called `describe` on a GroupBy object:

In [64]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [65]:
result.unstack()

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

Inside GroupBy, when we invoke a method like `describe`, it is actually just a shortcut for:

In [66]:
f = lambda x: x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

### 8.3.1 Suppressing the Group Keys

In the preceding examples, we see that the resulting object has a hierarchical index formed from the group keys along with the indexes of each piece of the original object. We can disable this by passing `group_keys=False` to `groupby`:

In [68]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


### 8.3.2 Quantile and Bucket Analysis

As we may recall from Chapter 6, pandas has some tools, in particular `cut` and `qcut`, for slicing data up into buckets with bins of your choosing or by sample quantiles. Combining these functions with `groupby` makes it convenient to perform bucket or
quantile analysis on a dataset. Consider a simple random dataset and an equal-length bucket categorization using `cut`:

In [69]:
frame = pd.DataFrame({'data1': np.random.randn(1000), 'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0    (-2.006, -0.275]
1     (-0.275, 1.455]
2    (-2.006, -0.275]
3    (-2.006, -0.275]
4    (-2.006, -0.275]
5     (-0.275, 1.455]
6    (-2.006, -0.275]
7    (-2.006, -0.275]
8     (-0.275, 1.455]
9     (-0.275, 1.455]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.743, -2.006] < (-2.006, -0.275] < (-0.275, 1.455] < (1.455, 3.185]]

The `Categorical` object returned by cut can be passed directly to groupby. So we could compute a set of statistics for the *data2* column like so:

In [71]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.743, -2.006]",-1.819281,1.459358,21.0,-0.135607
"(-2.006, -0.275]",-2.502894,2.691857,336.0,0.081408
"(-0.275, 1.455]",-3.238966,2.783379,561.0,-0.043395
"(1.455, 3.185]",-3.405221,2.451623,82.0,0.027281


These were equal-length buckets; to compute equal-size buckets based on sample quantiles, use `qcut`. We’ll pass `labels=False` to just get quantile numbers:

In [72]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.214795,2.572760,100.0,0.029250
1,-2.502894,2.231120,100.0,0.058782
2,-2.438451,2.691857,100.0,0.020570
3,-2.948539,2.608776,100.0,0.157264
4,-2.092404,2.393199,100.0,0.165632
5,-3.167170,2.395696,100.0,-0.018793
6,-2.726704,2.546539,100.0,-0.243724
7,-3.238966,1.990457,100.0,-0.152152
8,-2.321422,2.168805,100.0,-0.020428


### 8.3.3 Example: Filling Missing Values with Group-Specific Values

When cleaning up missing data, in some cases we will replace data observations using `dropna`, but in others we may want to impute (fill in) the null (NA) values using a fixed value or some value derived from the data. `fillna` is the right tool to
use; for example, here we fill in NA values with the mean:

In [73]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1    0.044454
2         NaN
3   -0.361586
4         NaN
5    0.053724
dtype: float64

In [74]:
s.fillna(s.mean())

0   -0.087803
1    0.044454
2   -0.087803
3   -0.361586
4   -0.087803
5    0.053724
dtype: float64

Suppose we need the fill value to vary by group. One way to do this is to group the data and use `apply` with a function that calls `fillna` on each data chunk. Here is some sample data on US states divided into eastern and western regions:

In [75]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -0.970577
New York     -0.021862
Vermont      -0.137873
Florida       0.559645
Oregon       -0.219171
Nevada       -0.899303
California    0.075624
Idaho        -0.451731
dtype: float64

Note that the syntax ['East'] * 4 produces a list containing four copies of the elements in ['East']. Adding lists together concatenates them. Let’s set some values in the data to be missing:

In [ ]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

In [ ]:
data.groupby(group_key).mean()

We can fill the NA values using the group means like so:

In [ ]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

In another case, we might have predefined fill values in our code that vary by group. Since the groups have a name attribute set internally, we can use that:

In [ ]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

## 8.4 Pivot Tables and Cross-Tabulation

### 8.4.1 Pivot Tables

A *pivot table* is a data summarization tool frequently found in spreadsheet programs and other data analysis software. It aggregates a table of data by one or more keys, arranging the data in a rectangle with some of the group keys along the rows and some along the columns. Pivot tables in Python with pandas are made possible through the `groupby` facility described in this chapter combined with reshape operations utilizing hierarchical indexing. DataFrame has a `pivot_table` method, and there is also a top-level `pandas.pivot_table` function. In addition to providing a convenience interface to groupby, `pivot_table` can add partial totals, also known as *margins*.

Returning to the tipping dataset, suppose you wanted to compute a table of group means (the default `pivot_table` aggregation type) arranged by *day* and *smoker* on the rows:

In [ ]:
tips.pivot_table(index=['day', 'smoker'])

This could have been produced with `groupby` directly. Now, suppose we want to aggregate only *tip_pct* and *size*, and additionally group by *time*. We’ll put *smoker* in the table columns and day in the rows:

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker')

We could augment this table to include partial totals by passing `margins=True`. This has the effect of adding All row and column labels, with corresponding values being the group statistics for all the data within a single tier:

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker', margins=True)

Here, the *All* values are means without taking into account smoker versus nonsmoker (the All columns) or any of the two levels of grouping on the rows (the All row).

To use a different aggregation function, pass it to `aggfunc`. For example, `count` or `len` will give you a cross-tabulation (count or frequency) of group sizes:

In [ ]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc=len, margins=True)

If some combinations are empty (or otherwise NA), we may wish to pass a `fill_value`:

In [ ]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'], columns='day', aggfunc='mean', fill_value=0)

See Table 8.2 for a summary of pivot_table methods.

<br><center>Table 8.2: *pivot_table* options</center>
<img src="Table 8.2.png" width="800px">

### 8.4.2 Cross-Tabulations: Crosstab

A cross-tabulation (or crosstab for short) is a special case of a pivot table that computes group frequencies. Here is an example:

In [ ]:
data1 = pd.DataFrame({'Sample': range(10), 'Nationality': ['USA', 'Japan', 'USA', 'Japan', 'Japan', 'Japan', 'USA',
                                                            'USA', 'Japan', 'USA'], 
                      'Handedness': ['Right-handed', 'Left-handed', 'Right-handed', 'Right-handed', 'Left-handed',
                                     'Right-handed', 'Right-handed', 'Left-handed', 'Right-handed', 'Right-handed',]})
data1

As part of some survey analysis, we might want to summarize this data by nationality and handedness. We could use `pivot_table` to do this, but the `pandas.crosstab` function can be more convenient:

In [ ]:
pd.crosstab(data1.Nationality, data1.Handedness, margins=True)

The first two arguments to `crosstab` can each either be an array or Series or a list of arrays. As in the tips data:

In [ ]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)